In [4]:
import pandas as pd

In [5]:
dir_path = "."

task_list_df = pd.read_csv(f"{dir_path}/3589453305.tsv", sep='\t', header=None, encoding='utf8')
task_list_df

FileNotFoundError: [Errno 2] No such file or directory: './3589453305.tsv'

In [ ]:
keywords = {
    "A7K3B": "Success",
    "X7R5T": "Ungrouped",
}

task_list_concat_df = pd.DataFrame()
task_list_concat_df['participant.label'] = task_list_df[4].str[-7:]
task_list_concat_df['participant.label'] = task_list_concat_df['participant.label'].str.extract(r'^l=(\d+)$').astype('int')
task_list_concat_df['yahoo_id'] = task_list_df[8]
task_list_concat_df['key_word'] = task_list_df[7]
task_list_concat_df['completion_status'] = task_list_concat_df['key_word'].map(keywords).fillna('Failed')
# task_list_concat_df['completion_status'] = task_list_concat_df['key_word']

print(task_list_concat_df.dtypes)
task_list_concat_df

success_keyword_participant_df = task_list_concat_df[task_list_concat_df['key_word'].map(keywords).notna()]
# success_keyword_participant_df = task_list_concat_df

In [11]:
pg_df = pd.read_csv(f"{dir_path}/all_apps_wide-2025-07-24.csv")
pg_concat_df = pd.DataFrame()
pg_concat_df['participant.label'] = pg_df['participant.label']
pg_concat_df['participant.payoff'] = pg_df.iloc[:,12]
pg_concat_df['PayPay_exchanged'] = pg_concat_df['participant.payoff'].apply(lambda x: x * 4 + 100)
pg_concat_df['PayPay_ceiled'] = pg_concat_df["PayPay_exchanged"].apply(lambda x: ((x+9) // 10)*10) # ホワイトリストのために1の位切り上げたリスト
pg_concat_df = pg_concat_df.dropna()
pg_concat_df = pg_concat_df.astype('int')
print(pg_concat_df.dtypes)
display(pg_concat_df)

pd.set_option('display.max_columns', None)

display(pg_df.head())
# pg_concat_df.to_csv("payoff_and_partictipantLabel.csv")

participant.label     int64
participant.payoff    int64
PayPay_exchanged      int64
PayPay_ceiled         int64
dtype: object


,participant.label,participant.payoff,PayPay_exchanged,PayPay_ceiled
0,10022,10,140,140
1,10024,26,204,210
2,10054,22,188,190
3,10035,10,140,140
4,10085,12,148,150
...,...,...,...,...
95,10009,10,140,140
96,10003,10,140,140
97,10086,10,140,140
98,10098,3,112,120


,participant.id_in_session,participant.code,participant.label,participant._is_bot,participant._index_in_pages,participant._max_page_index,participant._current_app_name,participant._current_page_name,participant.time_started_utc,participant.visited,participant.mturk_worker_id,participant.mturk_assignment_id,participant.payoff,participant.payoff_preferences_test,participant.payoff_risk_test,participant.payoff_airace,participant.payoff_crd,participant.decision_risk_test,participant.decision_permutation_index_risk_test,participant.result_risk_test,participant.finished,participant.reason_finished,participant.wait_page_arrival,participant.timeout_groups,participant.completion_status,participant.is_selected_PREV,participant.round_selected_PREV,participant.choice_selected_PREV,participant.row_selected_PREV,participant.X1_selected_PREV,participant.X2_selected_PREV,participant.P1_selected_PREV,participant.payment_amount_PREV,participant.is_selected_SOCPREF,participant.round_selected_SOCPREF,participant.choice_selected_SOCPREF,participant.payment_amount_SOCPREF,participant.role_soc_pref,participant.payment_amount_received,participant.payment_amount_given_to_other,participant.expiry,participant.allocations,participant.choice_order,session.code,session.label,session.mturk_HITId,session.mturk_HITGroupId,session.comment,session.is_demo,session.config.timeout_airace_comprehension_test_seconds,session.config.ungrouped_link,session.config.real_world_currency_per_point,session.config.name,session.config.transparency,session.config.timeout_pay_for_information_result_seconds,session.config.timeout_crd_comprehension_test_seconds,session.config.treatment,session.config.completion_link,session.config.group_size,session.config.information,session.config.round_decision_wait_page_timeout,session.config.target_lower_interval,session.config.uncertainty,session.config.timeout_redirect_to_prolific,session.config.endowment,session.config.timeout_airace_seconds,session.config.nb_tasks,session.config.participation_fee,session.config.target_higher_interval,session.config.information_cost,session.config.timeout_pledge_seconds,session.config.timeout_decision_link,session.config.timeout_crd_instructions_seconds,session.config.timeout_informed_consent_seconds,session.config.timeout_lost_focus_minutes,session.config.show_countdown_seconds,session.config.timeout_risk_test_seconds,session.config.timeout_airace_instructions_seconds,session.config.timeout_grouping_seconds,session.config.target,session.config.timeout_pay_for_information_seconds,session.config.no_consent_link,session.config.nb_rounds,session.config.timeout_link,session.config.timeout_pledge_results_seconds,session.config.timeout_grouping_minutes,session.config.kicked_out_link,session.config.collective_risk,session.config.timeout_results_view_seconds,session.config.timeout_final_experiment_results_view_seconds,session.config.timeout_survey_seconds,session.config.use_browser_bots,session.config.timeout_final_airace_results_view_seconds,session.config.timeout_crd_seconds,session.config.timeout_final_crd_results_view_seconds,session.config.opponent_timeout_link,session.config.timeout_general_instructions_seconds,session.config.timeout_focus_pledge_seconds,session.config.bonus_link,session.config.failed_test_link,session.waiting,session.grouped,crd_cost_information_consent.1.player.id_in_group,crd_cost_information_consent.1.player.role,crd_cost_information_consent.1.player.payoff,crd_cost_information_consent.1.player.signed_informed_consent,crd_cost_information_consent.1.player.signed_focus_pledge,crd_cost_information_consent.1.player.prolific_id,crd_cost_information_consent.1.group.id_in_subsession,crd_cost_information_consent.1.subsession.round_number,crd_cost_information_general_instr.1.player.id_in_group,crd_cost_information_general_instr.1.player.role,crd_cost_information_general_instr.1.player.payoff,crd_cost_information_general_instr.1.group.id_in_subsession,crd_cost_information_general_instr.1.subsessi

統合する

In [ ]:
marged_df = pd.merge(
    success_keyword_participant_df,
    pg_concat_df,
    on='participant.label',
    how='left'
)

display(marged_df)
marged_df.to_csv("yahooid-payoff.csv")

ホワイトリストの作成（下一桁切り上げたもので分類）

In [ ]:
import os
import csv
paypay_whitelist_df = marged_df[["yahoo_id", "PayPay_exchanged", "PayPay_ceiled"]]
whitelist_dict = {}
for idx, row in paypay_whitelist_df.iterrows():
    whitelist_dict[row["PayPay_ceiled"]] = []
for idx, row in paypay_whitelist_df.iterrows():
    whitelist_dict[row["PayPay_ceiled"]] += [row["yahoo_id"]]

os.makedirs(f"{dir_path}/whitelists", exist_ok=True)
for k in whitelist_dict.keys():
    yahooid_list = whitelist_dict[k]
    with open(f"{dir_path}/whitelists/20250724_{k}.txt", 'w') as f:
        writer = csv.writer(f)
        writer.writerow(yahooid_list)

print(sorted(whitelist_dict.keys()))

PayPayポイントのヒストグラム

In [ ]:
paypay_payoff = marged_df["PayPay_exchanged"].tolist()
paypay_task_num = set(paypay_payoff)
len(paypay_task_num)
import matplotlib.pyplot as plt
plt.hist(paypay_payoff, bins=100)
plt.show()

## 7/24実験結果について

グループわりを見つけ出す

In [ ]:
# グループの割り振り
group_set = set(pg_df['crd_cost_information.1.group.id_in_subsession'])
print(group_set)

participant_group = {}
participant_unfullgrouped = {}
for gl in group_set:
    tmp_df = pg_df[pg_df['crd_cost_information.1.group.id_in_subsession'] == gl]
    grouped_label = list(tmp_df['participant.label'])
    if len(grouped_label) != 4:
        participant_unfullgrouped[gl] = grouped_label
    else:
        participant_group[gl] = grouped_label

# グループが割り振られ、4人が集まったグループとその参加者ラベル
print(participant_group)

# グループが割り振られたが、何らかの理由で4人集まらなかったグループとその参加者ラベル
print(participant_unfullgrouped)


In [ ]:
# 4人グループが作れなかった人たちの原因
unfullgrouped_label = []
for v in participant_unfullgrouped.values():
    unfullgrouped_label += v

ungrouped_label_reason = [pg_df[pg_df['participant.label'] == ul]['participant.reason_finished']  for ul in unfullgrouped_label]
print(ungrouped_label_reason)

確認テストまで成功した人（successもしくはungrouped）

In [ ]:
success_label = pg_df[pg_df['participant.completion_status'] == 'success']['participant.label'].tolist()
ungrouped_label = pg_df[pg_df['participant.completion_status'] == 'ungrouped']['participant.label'].tolist()
label = pg_df['participant.label'].tolist()

print(success_label)
print(ungrouped_label)

寄付した額について

In [ ]:
# 寄付した額のカラムリスト
contribute_columns = [f'crd_cost_information.{i}.player.contribution' for i in range(1, 11)]

success_contribution = {}
for sl in success_label:
    row = pg_df[pg_df['participant.label'] == sl][contribute_columns]
    success_contribution[sl] = row.iloc[0].tolist()
print(success_contribution)

各グループと寄付額について

In [ ]:
round_column = [f"Round {i}" for i in range(1, 11)]

dfs = {}
for g, labels in participant_group.items():
    part_contrib = []
    part_index = []
    for l in labels:
        try:
            part_contrib.append(success_contribution[l])
            part_index.append(l)
        except KeyError as e:
            print(f"{l}のキーが存在しません。（success_contributionに含まれていない->success_labelにいない->途中離脱？）")
    # print(part_contrib)
    df = pd.DataFrame(part_contrib, columns=round_column, index=part_index)
    dfs[g] = df
    # display(df)

各グループの目標額を探し出す

In [ ]:
# ちゃんと4人集まったグループに限定する
group_target = {}
for g in participant_group.keys():
    group_target[g] = pg_df[pg_df['crd_cost_information.1.group.id_in_subsession'] == g]['crd_cost_information.1.group.target'].iloc[0]

print(group_target)

グラフ描画：累積貢献額（積み上げグラフ）

In [ ]:
import matplotlib.pyplot as plt
import os
"""
縦軸は累計貢献額、横軸がラウンド数
1.貢献額は積み立て式にする
2.そのグループの目標額を示す
"""
def TotalContribution(groups: list):
    os.makedirs(f"{dir_path}/total_contribution", exist_ok=True)
    for g in groups:
        df = dfs[g].cumsum(axis=1)
        df = df.T
        ax = df.plot(kind='bar', stacked=True)
        plt.axhline(y=group_target[g], color='red', linestyle='--', label='Target')
        plt.title(f"Group {g}")
        plt.ylim(0, 160)
        plt.ylabel("Total contribution (ECU)")
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"{dir_path}/total_contribution/Group_{g}.png")
        plt.show()
        break

TotalContribution(participant_group.keys())

グラフ描画：１ラウンドあたりの寄付額（折れ線グラフ）

In [ ]:
import os
def ContributionPerRound(groups: list):
    os.makedirs(f"{dir_path}/contribution_per_round", exist_ok=True)
    for g in groups:
        df = dfs[g]
        df = df.T
        ax = df.plot(kind='line', marker='o')
        plt.title(f"Group {g}")
        ax.set_yticks([0, 2, 4])
        plt.ylabel("Contribution per round (ECU)")
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"{dir_path}/contribution_per_round/Group_{g}")
        plt.show()
        break

ContributionPerRound(participant_group.keys())

目標達成したグループ、そうでないグループ

In [ ]:
target_success_group = []
target_failed_group = []

for g in participant_group.keys():
    group_mem = pg_df[pg_df['crd_cost_information.1.group.id_in_subsession'] == g].iloc[0]
    if group_mem['crd_cost_information.10.group.failed'] == 0:
        target_success_group.append(g)
    else:
        target_failed_group.append(g)

print(target_success_group, target_failed_group)

In [ ]:
TotalContribution(target_success_group)

各グループの参加者の提案額

In [ ]:
# dictに,labelをkeyとして、valueには[group, round1_pledge, round6_pledge]とする
participant_pledges = {}
for g in participant_group.keys():
    # 1ラウンドと6ラウンドの提案額
    pledges_df = pg_df[pg_df['crd_cost_information.1.group.id_in_subsession'] == g][['participant.label','crd_cost_information.1.player.pledge', 'crd_cost_information.6.player.pledge']]

    for index, row in pledges_df.iterrows():
        participant_pledges[int(row['participant.label'])] = [g, row['crd_cost_information.1.player.pledge'], row['crd_cost_information.6.player.pledge']]
    
print(participant_pledges)

グラフ描画：提案額も盛り込んだグラフ

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

for g, labels in participant_group.items():
    df = dfs[g].cumsum(axis=1)
    df = df.T
    ax = df.plot(kind='bar', stacked=True)
    plt.axhline(y=group_target[g], color='red', linestyle='--', label='Target')
    plt.legend()
    # 各参加者の提案額を加える
    for i, l in enumerate(labels):
        r1_pledge = participant_pledges[l][1]
        r6_pledge = participant_pledges[l][2]
        plt.plot([0, 5], [r1_pledge, r1_pledge], color=colors[i], linestyle='--', label=f'{l}_predges')
        plt.plot([5, 9], [r6_pledge, r6_pledge], color=colors[i], linestyle='--', label=f'{l}_predges')
    plt.title(f"Group {g}")
    plt.ylim(0, 160)
    plt.ylabel("Total contribution (ECU)")
    # plt.legend()
    plt.show()
    break

ブラックリストの作成

In [ ]:
blacklist = task_list_df[8].tolist()
print(blacklist)
import csv
with open(f"{dir_path}/blacklist.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(blacklist)